In [ ]:
!pip install --no-cache-dir --upgrade gdown
!gdown 1spSFY1yieMcjGJc989bbbEheNhegmEmR # challenge data
!7z e torob-data-challenge-2023_datafiles_v1.7z

In [ ]:
!gdown --id 1S86yEJLsHY3Q2a2NxoONLdBnxw5tDl5k # test dataset
!gdown --id 1--t-487fqCXP52zJpElbC5_DtQrMens7 # model weights
!gdown --id 1ZbkluwCQyN7cD-_bfyIF8DE1pTD6x5kM # test dataset

In [2]:
!pip install parsivar
!pip install hazm
!pip install python-negar
!pip install -q sentencepiece

!pip install -qU datasets
!pip install -qU transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 21.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/si

In [6]:
from collections import defaultdict, Counter

import csv
import json
import numpy as np
import pandas as pd

from parsivar import Normalizer
import hazm

from tqdm.notebook import tqdm_notebook

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import torch
from torch import nn

from datasets import Dataset, DatasetDict
from transformers import AdamW
from transformers import AutoTokenizer, AutoModel


In [8]:
test_file = pd.read_csv(
    "test_dataset_torob_v3_2.txt", encoding="utf-8", index_col="product_id"
)
test_file


,row_number,query,p_des,category_name,min_num_shops,max_num_shops,avg_num_shops,min_price,max_price,avg_price,mean_min_prices,mean_max_prices,mean_avg_prices,std_min_prices,std_max_prices,std_avg_prices
product_id,,,,,,,,,,,,,,,,
8823764,0,s ۲۰ fe,کاور سیلیکونی محافظ لنز دار مناسب برای گوشی مو...,کیف و کاور گوشی,4.0,6.0,5.258210,66000.0,69000.0,6.618826e+04,1.646523e+06,2.157218e+06,1.969483e+06,4.142260e+06,5.290222e+06,4.796780e+06
5812326,0,s ۲۰ fe,قاب سه بعدی مات سامسونگ S20 fe – کد573,کیف و کاور گوشی,1.0,1.0,1.000000,151200.0,151200.0,1.512000e+05,1.646523e+06,2.157218e+06,1.969483e+06,4.142260e+06,5.290222e+06,4.796780e+06
4182687,0,s ۲۰ fe,قاب و شاسی کامل سامسونگ Samsung Galaxy S20 FE,قاب، شاسی و درب گوشی,1.0,1.0,1.000000,374000.0,374000.0,3.740000e+05,1.646523e+06,2.157218e+06,1.969483e+06,4.142260e+06,5.290222e+06,4.796780e+06
4343057,0,s ۲۰ fe,گوشی موبایل سامسونگ مدل Samsung Galaxy S20 FE ...,گوشی موبایل,0.0,5.0,1.906094,13400000.0,15500000.0,1.457885e+07,1.646523e+06,2.157218e+06,1.969483e+06,4.142260e+06,5.290222e+06,4.796780e+06
5325921,0,s ۲۰ fe,قاب گوشی Galaxy S20 FE سامسونگ سیلیکونی اورجین...,کیف و کاور گوشی,1.0,1.0,1.000000,99000.0,105000.0,1.028974e+05,1.646523e+06,2.157218e+06,1.969483e+06,4.142260e+06,5.290222e+06,4.796780e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6722728,23139,ارزون ترین دوچرخه دوخترانه,دوچرخه دخترانه جونیور سایز 20 کد 04- Junior 04,دوچرخه کودک و نوجوان سایز 20,1.0,1.0,1.000000,3550000.0,3550000.0,3.550000e+06,3.810218e+06,4.012858e+06,3.862885e+06,2.188164e+06,2.214078e+06,2.188428e+06
7797092,23139,ارزون ترین دوچرخه دوخترانه,دوچرخه دخترانه الکس آناهیتا سایز 16 ANAHITA مدل,دوچرخه کودک سایز 16,2.0,3.0,2.990669,3750000.0,4050000.0,3.874961e+06,3.810218e+06,4.012858e+06,3.862885e+06,2.188164e+06,2.214078e+06,2.188428e+06
7870999,23139,ارزون ترین دوچرخه دوخترانه,دوچرخه بچه گانه دخترانه چیچک مدل پورت لاین سای...,دوچرخه کودک سایز 12,1.0,1.0,1.000000,2349900.0,2349900.0,2.349900e+06,3.810218e+06,4.012858e+06,3.862885e+06,2.188164e+06,2.214078e+06,2.188428e+06


In [9]:
ss = test_file.loc[5325921]
next(ss[ss["row_number"] == 0].iterrows())


(5325921, row_number                                                         0
 query                                                        s ۲۰ fe
 p_des              قاب گوشی Galaxy S20 FE سامسونگ سیلیکونی اورجین...
 category_name                                        کیف و کاور گوشی
 min_num_shops                                                    1.0
 max_num_shops                                                    1.0
 avg_num_shops                                                    1.0
 min_price                                                    99000.0
 max_price                                                   105000.0
 avg_price                                              102897.435897
 mean_min_prices                                       1646523.139535
 mean_max_prices                                       2157218.244186
 mean_avg_prices                                       1969483.167505
 std_min_prices                                        4142259.592916
 std_max_pr

In [10]:
class JsonFileIterator:
    def __init__(self, path):
        self.path = path
        self.f = open(path, "r")
        self.i = 0
        self.length = self.counter_lines()

    def __iter__(self):
        return self

    def __next__(self):
        line = self.f.readline()
        if not line:
            # End of file
            self.f.close()
            raise StopIteration
        self.i += 1
        return json.loads(line)

    def counter_lines(self):
        with open(self.path, "r") as f1:
            return sum(1 for _ in f1)

    def __len__(self):
        return self.length


In [11]:
tests = JsonFileIterator("/content/test-offline-data_v1.jsonl")


In [12]:
class MyNormalizer:
    def __init__(self):
        self.parsivar_normalizer = Normalizer(
            statistical_space_correction=True,
            half_space_char=" ",
            pinglish_conversion_needed=True,
        )
        self.hazm_normalizer = hazm.Normalizer(
            remove_extra_spaces=True,
            persian_numbers=True,
            persian_style=True,
            punctuation_spacing=False,
            remove_diacritics=True,
            affix_spacing=False,
            token_based=True,
        )

    def normilize(self, txt):
        return self.hazm_normalizer.normalize(
            self.parsivar_normalizer.normalize(
                txt.replace("\n", " ").replace("\u200c", " ").lower().strip()
            )
        )


In [13]:
class BaseModel(nn.Module):
    def __init__(self, c_model):
        super(BaseModel, self).__init__()
        self.bert = AutoModel.from_pretrained(c_model)
        self.fc = nn.Linear(self.bert.config.hidden_size, 256)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(256, 1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        out = self.fc(output[1])
        return self.fc2(self.relu(out))


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
c_model = "HooshvareLab/bert-fa-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(c_model)
model = BaseModel(c_model)
model.to(device)
model.eval();

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [ ]:
model.load_state_dict(torch.load("./torobv3_2-ParsBERT-model_1-epoch_2.w"), strict=True)
model.to(device);

In [ ]:
tests = JsonFileIterator("./test-offline-data_v1.jsonl")


In [ ]:
normalizer = MyNormalizer()

rank_lines = []

for irow, t in enumerate(tqdm_notebook(tests)):
    query = normalizer.normilize(t["raw_query"])
    ranked_list = []
    input_ids_List = []
    attention_mask_list = []
    token_type_ids_list = []
    e_products = []
    for res in t["result_not_ranked"]:
        try:
            ss = test_file.loc[res]
            if type(ss) != pd.core.series.Series:
                product = next(ss[ss["row_number"] == irow].iterrows())[1]
            else:
                product = ss
    
            p_des = normalizer.normilize(product["p_des"])
            category = product["category_name"]
            max_shop = int(np.log2(int(product["max_num_shops"]) + 1))
            popularity = f"popularity is {normalizer.normilize(str(max_shop))}"
            price_level = f"price_level is {normalizer.normilize(product["price_level"])}"
            encoded_text = tokenizer(
                query,
                category + " " + popularity + " " + " " + price_level + " " + p_des,
                truncation=True,
                max_length=512,
                return_tensors="pt",
            )

            e_products.append(res)
            input_ids = encoded_text["input_ids"]
            attention_mask = encoded_text["attention_mask"]
            token_type_ids = encoded_text["token_type_ids"]
            input_ids_List.append(input_ids.squeeze(0))
            attention_mask_list.append(attention_mask.squeeze(0))
            token_type_ids_list.append(token_type_ids.squeeze(0))
        except:
            print(res)
            print(irow)

    input_ids_List = torch.nn.utils.rnn.pad_sequence(
        input_ids_List, batch_first=True, padding_value=0.0
    )
    attention_mask_list = torch.nn.utils.rnn.pad_sequence(
        attention_mask_list, batch_first=True, padding_value=0.0
    )
    token_type_ids_list = torch.nn.utils.rnn.pad_sequence(
        token_type_ids_list, batch_first=True, padding_value=0.0
    )
    with torch.no_grad():
        preds = model(
            input_ids=input_ids_List.to(device),
            attention_mask=attention_mask_list.to(device),
            token_type_ids=token_type_ids_list.to(device),
        )
    rank_lines.append(list(zip(e_products, preds.view(-1, 1))))
    if irow % 1000 == 0:
        with open("output.csv", "w", newline="") as csvfile:
            writer = csv.writer(csvfile)
            for row in rank_lines:
                writer.writerow(
                    [x[0] for x in sorted(row, key=lambda x: x[1].item())[::-1]]
                )

  0%|          | 0/23140 [00:00<?, ?it/s]

6142860
336
4276004
366
8432823
596
6510223
692
3466388
896
7257827
1064
649147
1064
2234373
1064
3564896
1615
3556535
2470
3455606
2477
4671019
2477
9343304
2480
9343304
2481
7217184
2700
6715733
2911
6715733
2967
3912044
3370
5443918
3440
9324045
3464
6417072
3464
9324045
3465
3344067
3630
1777347
3630
8146623
3821
5512054
3831
9343304
3847
3455606
3989
4671019
3989
8902003
4113
8558914
4478
3033365
4478
437779
4478
3156885
4478
4972560
4732
5443918
4879
542363
5088
3564896
5285
1352387
5349
6141008
5427
7965301
6020
335960
6279
1548100
6279
9343304
6439
4794552
6442
671353
6543
6073672
6545
933843
6879
4711876
7331
6264349
7331
6899722
7431
7854568
7431
3761024
7824
3458228
8012
970917
8256
4276004
8515
4877389
8608
745161
8794
5608714
9254
1854656
9254
4429711
9254
6715733
9649
6420096
9912
1548100
10724
335960
10724
3458532
11347
925359
12058
2353939
12075
5676242
12115
9324045
12418
3684402
12578
7532679
13259
5314220
14213
9239475
14255
6715733
14687
2566958
14937
4671019
14970


In [16]:
with open(
    "/content/drive/MyDrive/output_error_v4_5_750k.csv", "w", newline=""
) as csvfile:
    writer = csv.writer(csvfile)
    for row in rank_lines:
        writer.writerow([x[0] for x in sorted(row, key=lambda x: x[1].item())[::-1]])


In [17]:
tests = JsonFileIterator("/content/test-offline-data_v1.jsonl")
predict_list = []
with open("/content/drive/MyDrive/output_error_v4_5_750k.csv") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=",")
    for row in csv_reader:
        predict_list.append(row)
test_list = JsonFileIterator("/content/test-offline-data_v1.jsonl")
for i, products in enumerate(tqdm_notebook(test_list)):

    tp = set([x for x in products["result_not_ranked"]])
    pp = set([int(x) for x in predict_list[i]])
    differ_pid = tp - pp
    if len(differ_pid) > 0:
        [predict_list[i].append(x) for x in list(differ_pid)]
        print(i)
        print(differ_pid)


  0%|          | 0/23140 [00:00<?, ?it/s]

336
{6142860}
366
{4276004}
596
{8432823}
692
{6510223}
896
{3466388}
1064
{649147, 2234373, 7257827}
1615
{3564896}
2470
{3556535}
2477
{4671019, 3455606}
2480
{9343304}
2481
{9343304}
2700
{7217184}
2911
{6715733}
2967
{6715733}
3370
{3912044}
3440
{5443918}
3464
{6417072, 9324045}
3465
{9324045}
3630
{3344067, 1777347}
3821
{8146623}
3831
{5512054}
3847
{9343304}
3989
{4671019, 3455606}
4113
{8902003}
4478
{8558914, 437779, 3033365, 3156885}
4732
{4972560}
4879
{5443918}
5088
{542363}
5285
{3564896}
5349
{1352387}
5427
{6141008}
6020
{7965301}
6279
{335960, 1548100}
6439
{9343304}
6442
{4794552}
6543
{671353}
6545
{6073672}
6879
{933843}
7331
{4711876, 6264349}
7431
{7854568, 6899722}
7824
{3761024}
8012
{3458228}
8256
{970917}
8515
{4276004}
8608
{4877389}
8794
{745161}
9254
{1854656, 5608714, 4429711}
9649
{6715733}
9912
{6420096}
10724
{335960, 1548100}
11347
{3458532}
12058
{925359}
12075
{2353939}
12115
{5676242}
12418
{9324045}
12578
{3684402}
13259
{7532679}
14213
{5314220}
1

In [18]:
with open(
    "/content/drive/MyDrive/prediction_v4_5_750k.csv", "w", newline=""
) as csvfile:
    writer = csv.writer(csvfile)
    for row in predict_list:
        writer.writerow(row)


In [ ]:
ss = test_file.loc[res]
product = next(ss[ss["query"] == query].iterrows())[1]
p_des = normalizer.normilize(product["p_des"])
category = product["category_name"]
max_shop = int(np.log2(int(product["max_num_shops"]) + 1))
popularity = "popularity" + normalizer.normilize(str(max_shop))
avg_price = product["avg_price"]
std_avg_prices = product["std_avg_prices"]
mean_avg_prices = product["mean_avg_prices"]
if (avg_price != None) and not np.isnan(avg_price):
    price_level = "price is " + normalizer.normilize(
        str(int((((avg_price - mean_avg_prices) / std_avg_prices) + 2) * 5))
    )
else:
    price_level = "price is none"

encoded_text = tokenizer(
    query,
    category + " " + popularity + " " + " " + price_level + " " + p_des,
    truncation=True,
    max_length=512,
    return_tensors="pt",
)


In [ ]:
ss = test_file.loc[8823764]
product = next(ss[ss["query"] == query].iterrows())[1]

p_des = normalizer.normilize(product["p_des"])
category = product["category_name"]
max_shop = int(np.log2(int(product["max_num_shops"]) + 1))
popularity = "popularity" + normalizer.normilize(str(max_shop))
avg_price = product["avg_price"]
std_avg_prices = product["std_avg_prices"]
mean_avg_prices = product["mean_avg_prices"]
if (avg_price != None) and not np.isnan(avg_price):
    price_level = "price is " + normalizer.normilize(
        str(int((((avg_price - mean_avg_prices) / std_avg_prices) + 2) * 5))
    )
else:
    price_level = "price is none"


{'query': 's ۲۰ fe', 'p_des': 'کاور سیلیکونی محافظ لنز دار مناسب برای گوشی موبایل سامسونگ  Galaxy S20 FE S20FE Cover For قاب Fe s20 اصلی اورجینال گارد Silicone for fe Samsung مدل پاک کنی) (اصلی', 'category_name': 'کیف و کاور گوشی', 'min_num_shops': 4.0, 'max_num_shops': 6.0, 'avg_num_shops': 5.258210230741057, 'min_price': 66000.0, 'max_price': 69000.0, 'avg_price': 66188.25540227079, 'mean_min_prices': 1646523.1395348837, 'mean_max_prices': 2157218.2441860465, 'mean_avg_prices': 1969483.1675053996, 'std_min_prices': 4142259.592916288, 'std_max_prices': 5290221.597737699, 'std_avg_prices': 4796780.465283441}
کاور سیلیکونی محافظ لنز دار مناسب برای گوشی موبایل سامسونگ  Galaxy S20 FE S20FE Cover For قاب Fe s20 اصلی اورجینال گارد Silicone for fe Samsung مدل پاک کنی) (اصلی


In [ ]:
next(ss[ss["query"] == query].iterrows())[1]["query"]


's ۲۰ fe'

In [ ]:
ss[ss["query"] == query]


,query,p_des,category_name,min_num_shops,max_num_shops,avg_num_shops,min_price,max_price,avg_price,mean_min_prices,mean_max_prices,mean_avg_prices,std_min_prices,std_max_prices,std_avg_prices
product_id,,,,,,,,,,,,,,,
8823764,s ۲۰ fe,کاور سیلیکونی محافظ لنز دار مناسب برای گوشی مو...,کیف و کاور گوشی,4.0,6.0,5.25821,66000.0,69000.0,66188.255402,1.646523e+06,2.157218e+06,1.969483e+06,4.142260e+06,5.290222e+06,4.796780e+06
8823764,s ۲۰ fe,کاور سیلیکونی محافظ لنز دار مناسب برای گوشی مو...,کیف و کاور گوشی,4.0,6.0,5.25821,66000.0,69000.0,66188.255402,2.635517e+06,3.572276e+06,3.225102e+06,5.165618e+06,6.731951e+06,6.054369e+06
8823764,s ۲۰ fe,کاور سیلیکونی محافظ لنز دار مناسب برای گوشی مو...,کیف و کاور گوشی,4.0,6.0,5.25821,66000.0,69000.0,66188.255402,1.699585e+06,2.177649e+06,1.999767e+06,4.264132e+06,5.306488e+06,4.855224e+06
8823764,s ۲۰ fe,کاور سیلیکونی محافظ لنز دار مناسب برای گوشی مو...,کیف و کاور گوشی,4.0,6.0,5.25821,66000.0,69000.0,66188.255402,2.414648e+06,3.264381e+06,2.945438e+06,4.945934e+06,6.449555e+06,5.802985e+06
8823764,s ۲۰ fe,کاور سیلیکونی محافظ لنز دار مناسب برای گوشی مو...,کیف و کاور گوشی,4.0,6.0,5.25821,66000.0,69000.0,66188.255402,2.449656e+06,3.338287e+06,3.009370e+06,4.933941e+06,6.432046e+06,5.786546e+06


In [ ]:
with open("/content/drive/MyDrive/output_v4_750f.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    for row in rank_lines:
        writer.writerow([x[0] for x in sorted(row, key=lambda x: x[1].item())[::-1]])


In [ ]:
with open("/content/drive/MyDrive/output_v3_750f.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    for row in rank_lines:
        writer.writerow([x[0] for x in sorted(row, key=lambda x: x[1].item())[::-1]])
